# Assignment 4

***

*Peer-to-peer*

***

The last major part of your blockchain in Python boils down to the **management of communication between the different actors in the network** of this blockchain. Now that you have defined all the rules governing the stability and security of your blockchain on your local machine, you still need to manage the sending and receiving of information with others on the network, who will probably try to cheat you with blocks that turn the system to their advantage.

Remember the concepts discussed in class: a **node** is an entity operating on the communication network and interacting with other nodes. For our blockchain, a node ensures the management of the blockchain locally and shares it on the network, while providing easy access to any user who connects to it. Its tasks include:
* Keeping a copy of the blockchain
* Allowing a "human" user to consult or feed this local blockchain with new information
* Sharing the new information received with other nodes on the network
* Filtering information from other nodes to destroy what is not legal

Don't worry, you don't have to implement the purely *networking* part of peer-to-peer. In this assignment, you will work with a class that **simulates a peer-to-peer network**. You will only need to implement the four behaviors described above.

In [1]:
# Mandatory cell, please execute it

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

### Node

The term **node** is not specific to blockchain. Generally, in any communication network, a "**node**" describes an entity that **sends messages to other nodes** and **receives messages from other nodes**.

With this Assignment 4, you have downloaded a new folder "**scripts**" which contains a folder "**network**". You must place this "network" folder in the same location as your "helpers" folder. Your folder architecture should be as follows:

* SomeFolder
    * scripts
        * helpers
            * ...
        * network
            * \_\_init\_\_.py
            * dummy_node.py
            * node_base.py
        * Your Python scripts...
    * notebooks
        * **This notebook**
    * ...

This folder contains a `Node` class that simulates a network node. You can access it by writing `from network import Node`, and then making your class inherit from `Node`. If you do this, you must do two things in your class:
* Call the constructor of `Node` by doing `super().__init__(nodeIdentifier)` where `nodeIdentifier` is an identifier of your choice, to distinguish the nodes in the network. Take inspiration from the example in the following cell as well as your work on the `Block` class which inherits from `Certificate` and passes to the constructor of `Certificate` the public key of the issuer.
* Implement the function `receive_object_from_node(obj, nodeIdentifier)`, which is used to receive objects sent to you by other nodes in the network (again, see the following cell for an example).

Furthermore, you can also use the Node class to send objects to other nodes in the network, via two functions:
* `self.send_object_to_node(object, nodeIdentifier)`: Sends the Python object `object` to the node whose identifier is `nodeIdentifier`. The `receive_object_from_node` function implemented in the node with the identifier `nodeIdentifier` will then be called with the parameters `object` and `nodeIdentifier` in order.
* `self.broadcast_object(object)`: Identical to the previous function, but this time it sends the Python object `object` to all nodes in the network (except ourselves).

You can certainly rewrite them in your class that inherits from `Node`, but don't forget to call their parent class by doing `super().send_object_to_node(object, nodeIdentifier)` or `super().broadcast_object(object)`, otherwise the network behavior will be broken (see cell just after).

**The following cells illustrate an example of use**.

In [2]:
from network import Node

# Creating a mini class "ExampleNode", which inherits the "Node" class.
# You will need to do the same for your "BlockchainNode" class.
# The code snippet between the two lines will probably be the same for your "BlockchainNode" class.
# The rest is just here for demonstration purposes.

class ExampleNode(Node):
    
    #———————————————————————————————————————————————————————————————————————————————————————————————————
    
    # We call "Node"'s constructor to register the node identifier
    
    def __init__(self, nodeIdentifier):
        super().__init__(nodeIdentifier)
        
    # The following function serves as a receiver for any data sent by other node to us
        
    def receive_object_from_node(self, obj, senderNodeIdentifier):
        if isinstance(obj, int):
            self.print(f"\"{senderNodeIdentifier}\" sent me the integer \"{obj}\"")
        elif isinstance(obj, str):
            self.print(f"\"{senderNodeIdentifier}\" sent me the string \"{obj}\"")
        elif isinstance(obj, bool):
            self.print(f"\"{senderNodeIdentifier}\" sent me the boolean \"{obj}\"")
        else:
            self.print(f"I do not know the type of data \"{senderNodeIdentifier}\" just sent me...")
            
    #———————————————————————————————————————————————————————————————————————————————————————————————————
        
    # Small print function
        
    def print(self, message):
        print(f'[{self.nodeIdentifier}] {message}')
            
    # Here we CAN (not mandatory) override the send_object_to_node function to include a pretty print
            
    def send_object_to_node(self, obj, nodeIdentifier):
        self.print(f"I am sending the data \"{obj}\" to \"{nodeIdentifier}\"")
        super().send_object_to_node(obj, nodeIdentifier)
            
    # ...here too
        
    def broadcast_object(self, obj):
        self.print(f"I am sending the data \"{obj}\" to EVERYBODY!")
        super().broadcast_object(obj)
        
# We reset the whole network to avoid duplicate nodes
Node.reset_network()
        
# Creating 3 nodes on the network
exampleNode1 = ExampleNode("Example Node 1")
exampleNode2 = ExampleNode("Example Node 2")
exampleNode3 = ExampleNode("Example Node 3")

```"Example Node 1"``` sends the integer ```42``` to ```"Example Node 2"```

In [3]:
exampleNode1.send_object_to_node(42, "Example Node 2")

[Example Node 1] I am sending the data "42" to "Example Node 2"
[Example Node 2] "Example Node 1" sent me the integer "42"


```"Example Node 2"``` sends the string ```"bonjour"``` to ```"Example Node 3"```

In [4]:
exampleNode2.send_object_to_node("bonjour", "Example Node 3")

[Example Node 2] I am sending the data "bonjour" to "Example Node 3"
[Example Node 3] "Example Node 2" sent me the string "bonjour"


Notice how the type of data is recognized by the receiver, as per how we implemented it in the cell.

***

```"Example Node 3"``` sends the **decimal number** ```3.14159``` (Pi) to everybody.

In [5]:
exampleNode3.broadcast_object(3.14159)

[Example Node 3] I am sending the data "3.14159" to EVERYBODY!
[Example Node 3] I am sending the data "3.14159" to "Example Node 1"
[Example Node 1] I do not know the type of data "Example Node 3" just sent me...
[Example Node 3] I am sending the data "3.14159" to "Example Node 2"
[Example Node 2] I do not know the type of data "Example Node 3" just sent me...


### Blockchain Nodes

In the same way that nodes exchange Python objects in our above example, nodes in the blockchain network exchange information (blocks and certificates) among themselves.

In our current configuration, which you have obtained after completing Assignments 2 and 3, we are going to implement the `BlockchainNode` class. The goal of this class is to combine the blockchain and the consensus algorithm to, on the one hand, participate in the creation of blocks on the blockchain, and on the other hand, filter out bad additions and modifications to it. All this takes place on a network of nodes.

Your class should inherit from `Node` to enable communication with other nodes in the network. Again, refer to the example with the `ExampleNode` class above.

***

<font color="7777aa">In "scripts", create the file `node.py`, in which you must implement the `BlockchainNode` class. In its constructor, in addition to initializing the `Node` class, you need to take care of initializing each component of the node:
* a `wallet` **(which you will pass as an argument to the constructor)** to have a digital identity on the blockchain, as well as to sign and forge blocks,
* a consensus algorithm `consensusAlgorithm` **(which you will also pass as an argument to the constructor)**,
* a `blockchain` that will be synchronized with that of other nodes. **Do not pass it as an argument; it must be created from scratch by the node.**</font>

> <details><summary><strong>Click here for help</strong></summary>As a node identifier, you could, for example, use the public key of the node's wallet, but it's really up to you...</details>

In [6]:
from wallet import Wallet
from node import BlockchainNode
from proof_of_stake import ProofOfStake

# First node's wallet, also the default forger
defaultForgerWallet = Wallet()

# We use Proof-of-Stake (PoS) for our blockchain
proofOfStake = ProofOfStake(defaultForgerWallet.publicKey)

Node.reset_network()  # We completely reset the network
node1 = BlockchainNode(defaultForgerWallet, proofOfStake)  # Creating first node

assert node1.wallet.publicKey == proofOfStake.defaultForgerPublicKey # Node's wallet is the default forger
assert node1.consensusAlgorithm == proofOfStake                      # It knows we are using PoS
assert node1.nodeIdentifier                                          # It has a node identifier
assert node1.blockchain.is_legit()                                   # Its blockchain exists and is legit

"Success!"

'Success!'

For the rest of this Assignment, we'll need 4 humans that are interacting with the blockchain.

In [7]:
Alice = Wallet()    # Alice's wallet
Bob = Wallet()      # Bob's wallet
Charlie = Wallet()  # Charlie's wallet
Delphine = Wallet() # Delphine's wallet

humans = [Alice, Bob, Charlie, Delphine]  # A list with our 4 humans

# We will keep track of each certificates
certificates = {
    Alice: [],
    Bob: [],
    Charlie: [],
    Delphine: []
}

### Certificate Box

As we discussed in the course on consensus algorithms, certificates are only added to a block at the time of forging that block. Until then, they need to be stored somewhere.

It is common practice for each node to keep a "certificate box" in which it places the certificates given to it (whether they come from a human or another node) and only empties it when forging a block.

Remember: **certificates are always unique in the blockchain!!** Two certificates that have the same hash are strictly identical (same issuer, same creation date, and same data). Normally, issuing the same certificate twice in a blockchain is prohibited, and if this happens, it is probably a duplicate and therefore should be discarded.

In Python, there is a data structure that acts a bit like a list but refuses duplicates: this is the `set()`. When you add an object `object` to a set `dummySet` using `dummySet.add(object)`, Python first checks if this object already exists in `dummySet`, and if so, adds nothing. Similarly, you can remove an object with `dummySet.remove(object)` and check if an object is present with `if object in dummySet: ...`. **Be aware though, sets are not indexed like lists, which means you cannot get the i-th element using** `dummySet[i]`**.**

Thus, you can be sure that all your objects in `dummySet` are present in only **one single copy**.

A set works with hashes to check the presence or absence of a Python object in itself. When you want to add, remove, or verify the presence of an object in the set, it compares the hash of the object with the hashes of the objects present in the set. This is convenient because our certificates already have a `hash()` function and an `equals(otherCertificate)` function. We just need to make Python understand that it should use these functions for the set.

***

For hashing any object, Python has a generic function `hash(object)` that takes an object as input and returns an **integer**. In reality, `hash(object)` is strictly equivalent to `object.__hash__()`, and therefore every Python object has a `__hash__()` function.

<font color="7777aa">In your `Certificate` class, rewrite the `__hash__()` function so that it uses your `hash()` function defined in Tutorial 2. **Remember that it must return an integer!! Not a hexadecimal string...**</font>

> <details><summary><strong>Click here for help</strong></summary>Remember that to convert a hexadecimal string to an integer, you can use the function <code>int(hashString, 16)</code></details>

In [8]:
from certificate import Certificate

# Two certificates created at the exact same time by the same person : they are identical
dummyCertificate1 = Certificate(Alice.publicKey)
dummyCertificate2 = Certificate(Alice.publicKey)

# ...However, it is possible that successive calls to timestamp.now() yield timestamps 1 milliseconds apart...
dummyCertificate2.timestamp = dummyCertificate1.timestamp

assert isinstance(hash(dummyCertificate1), int)            # The hash is an integer
assert hash(dummyCertificate1) == hash(dummyCertificate2)  # Both certificates have the same hash (using Python's hash function)

f"Success! Certificate's hash : {hash(dummyCertificate1)}"

"Success! Certificate's hash : 1722716824715417887"

Similar to the hashing process, when you perform `object1 == object2` in Python, what actually happens behind the scenes is that Python executes `object1.__eq__(object2)` (assuming neither of the objects is `None`).

<font color="7777aa">In your `Certificate` class, rewrite the `__eq__(otherCertificate)` function so that it uses your `equals(otherCertificate)` function defined in Tutorial 2.</font>

In [9]:
assert dummyCertificate1 == dummyCertificate2  # Both certificates are different but considered identical

"Success!"

'Success!'

Now, we can use `set`s properly with our certificates.

In [10]:
dummySet = set()

# We try to add both identical certificates to the set
dummySet.add(dummyCertificate1)
dummySet.add(dummyCertificate2)

assert len(dummySet) == 1              # The set discarded the "duplicate" certificate
assert dummyCertificate1 in dummySet   # The first certificate is inside the set
assert dummyCertificate2 in dummySet   # The second certificate too since it is "identical"

dummySet.remove(dummyCertificate2)  # Here we remove the second certificate (which was never here in the first place since it got discarded)

assert len(dummySet) == 0   # We nonetheless end up with an empty set

"Success!"

'Success!'

Returning to your certificate box, using a set will give you exactly the desired behavior: a collection of unique certificates. Therefore, you can now equip your node with a certificate box.

***

<font color="7777aa">Add to your `BlockchainNode` constructor the initialization of the field `self.__certificateBox = set()`. The double underscores `__` make the field private, in the same way as you did with the private key of the wallet, because we do not want to give users full control over the addition and removal of certificates (especially as we do not want to see illegal certificates in there).</font>

In [11]:
Node.reset_network()                                        # We reset the network
node1 = BlockchainNode(defaultForgerWallet, proofOfStake)   # We connect 1 node on the network

assert hasattr(node1, '_BlockchainNode__certificateBox')    # Sometimes Python just looks awful...

"Success!"

'Success!'

Now that we have a way to receive certificates that other actors (humans or nodes) send us, we can start to design the behavior of the node.

As we saw in class, since the blockchain is fixed at a node, the only information that circulates between nodes are certificates and blocks, so that everyone has an up-to-date version of the blockchain. We could even go further and only circulate blocks (since blocks contain certificates, killing two birds with one stone), but the risk is that some certificates never end up in the blockchain because their owner is never the forger. So, we will circulate both.

But let's start with the basics: what happens when the node receives a certificate from a human?

***

<font color="7777aa">Write in the `BlockchainNode` class a function `new_certificate(certificate)` that manages the addition of new certificates to the node. The process is as follows:
* Verify that the certificate is honest and that it should indeed be added to our certificate box. This is the validation step. If yes, add it to the box and continue, otherwise stop here because nothing has changed.
* If our box contains more than an arbitrary number of certificates (in this Assignment, let's take **5**), we need to check if it's our turn to forge the next block. If yes, we forge it with all the certificates in the box, add it to our blockchain, and send it to everyone.
* Otherwise, send the certificate to everyone to make sure everyone is up to date.</font>

**Remember that the blockchain hates duplicates!!! A certificate should never end up in double inside it.**

> <details><summary><strong>Click here for help 1</strong></summary>The validation step involves 3 tests. For one of these tests, I recommend creating a function <code>contains_certificate(certificate)</code> in the <code>Blockchain</code> class that returns <code>True</code> if the certificate is present in the blockchain and <code>False</code> otherwise. You might reuse it later.</details>

> <details><summary><strong>Click here for help 2</strong></summary>To know if you are the next forger, you have precisely coded in Assignment 3 the function <code>is_next_block_forger_legit(blockList, block)</code> in Proof-of-Stake (and PoW if you did the bonus). So, create a block as if you were going to forge it, and use this function on it: it will tell you whether you can add it to your blockchain and share it with everyone as a forger or not.</details>

> <details><summary><strong>Click here for help 3</strong></summary>Forging a block means signing it, adding it to your blockchain, and then sending it to everyone. You have the duty to give it the correct index, the correct parent, and to put yourself as the issuer of this block.</details>

In [12]:
# We will generate 6 certificates, 1 being fraudulous (Charlie's) and add them twice each to the node. 

for i in range(6):
    selectedHuman = humans[i % len(humans)]
    if selectedHuman == Charlie:
        certificate = Certificate(Alice.publicKey)
    else:
        certificate = Certificate(selectedHuman.publicKey)
    certificate.timestamp = i
    selectedHuman.sign(certificate)
    certificates[selectedHuman].append(certificate)
    node1.new_certificate(certificate)
    node1.new_certificate(certificate)
    
# To push our system further, let's add Bob's one more time

node1.new_certificate(certificates[Bob][0])

# Without any duplicity and/or legality tests, we would normally have 13 certificates inside the box
# And thus, 3 blocks (containing each 0, 5 and 5 certificates) inside our blockchain
# Except invalid certificates are discarded, which means Charlie's (in double)
# We are short 2 certificates, down to 11
# Then we discard duplicates (2 for Alice, 3 for Bob and 1 for Delphine)
# We end up with 5 certificates which means 2 blocks in the blockchain and no certificate in the box

if hasattr(node1.blockchain, 'display'):
    node1.blockchain.display()

assert len(node1.blockchain.blockList) == 2   # We have 2 blocks since we only have 5 different legit certificates
assert len(node1._BlockchainNode__certificateBox) == 0  # No pending certificate

# Charlie's invalid certificate is not in the blockchain
assert certificates[Charlie][0] not in node1.blockchain.blockList[0].certificateList

"Success!"

Block 0:
{'type': 'Block', 'issuerPublicKey': '0000000000', 'timestamp': 0, 'hash': '998cc8c77c2c1f9b2cc11de66dbf5764787d3a357207cd4e76cb062fcf4e9552', 'signature': '', 'index': 0, 'parent': '0000000000000000000000000000000000000000000000000000000000000000', 'certificates': [], 'nonce': 0}
Block 1:
{'type': 'Block', 'issuerPublicKey': '3082012230', 'timestamp': 1738944127553, 'hash': 'ed7c89134f207575178b84111da528c17d324e5158617c7434f989a31ec274f1', 'signature': '754958d11e495a151eceff272122e6b2a9edec3585166ee0a538cd5c659e289e97b17b9ea90ca63ab3b66fff9737cc519c49cfc14d6bfe5363f8e7c9c6185bbe06365ad50f7f73f907f54526348a85fa48e1bf32ea11dcefec9a1dc05f7733e1d91d0fe889f3240ffd106ea56dacfaa62eb5390c9520ff928c34e5f32721211c5a1c37214562043e57073339d679c07f875806b7e9f3993099e614e33d7b3ae439bf418e602f932e0f38e210b6c8251aa5aa83a4c88a9620b1c974fe58cb8a6bbd23bac09d99e4ec32ba442ce53166e8a5c9555d1160b6e8e04eb897504ed3012c2a9face5377d4a1cafe7bc1d5bdcf93bce5f4f8643faccb5bff415e33d091b', 'index': 1, 'par

'Success!'

Our humans are now capable of providing certificates to our nodes using the function we just created.

Generally, this is the only thing humans are allowed to give to the node. They can also consult the blockchain, but nothing more.

Let's now focus on the case of other nodes: in the network, we will receive either blocks or certificates from other nodes (see for yourself your function `new_certificate(certificate)`: it sends certificates and blocks).

Receiving a certificate from a node is fundamentally identical to receiving it from a human since in both cases we assume it's a new certificate (otherwise we simply ignore it).

So, we really only have to write a function that handles the management of blocks arriving at the node.

***

Write in the `BlockchainNode` class a function `new_block(block)` that manages the addition of new blocks to the node. You must follow the same procedure as for certificates, namely validating the addition of the block to the end of your blockchain, and then broadcasting it to all your peers if the block is validated.

We will assume that in our simulations, blocks are always added in the right order and that the case where we would receive two different but valid blocks, and therefore have to keep the longest blockchain (as seen in class), never happens. Simply refuse any block that is not directly addable to the blockchain in its current state.

**Be careful, you might be sent a block containing a certificate that you already have pending at your place...**

> <details><summary><strong>Click here for help 1</strong></summary>The validation step involves exactly 6 tests. What are they? (One of them is harder to find, see help 2.)</details>

> <details><summary><strong>Click here for help 2</strong></summary>Remember that the blockchain hates déjà vu...</details>

> <details><summary><strong>Click here for help 3</strong></summary>Once the block is validated and added to your blockchain, before passing it on to others, there is a bit of cleaning up to do and then only it might be your turn to contribute to the edifice...</details>

In [13]:
from random import random
from block import Block

# To be able to reexecute this cell, let's set the blockchain back to its previous state

if len(node1.blockchain.blockList) > 2:
    node1.blockchain.blockList = node1.blockchain.blockList[:2]

# The following loop tries many different reasons to reject a block
# Over the 20 blocks we try to forge, only 2 are valid

actors = [Alice, node1.wallet]
    
for i in range(20):
    blockIssuer = actors[min(1, (i * 3) % 6)]
    blockIndexInBlockchain = 2 + (i % 5)
    blockParentBlockHash = node1.blockchain.blockList[i % len(node1.blockchain.blockList)].hash()
    blockCertificateList = []
    for j in range(4 + i % 2):
        selectedHuman = humans[j % len(humans)]
        certificate = Certificate(selectedHuman.publicKey)
        selectedHuman.sign(certificate)
        blockCertificateList.append(certificate)
    block = Block(blockIssuer.publicKey, blockIndexInBlockchain, blockParentBlockHash, blockCertificateList)
    blockIssuer.sign(block)
    node1.new_block(block)
    
assert len(node1.blockchain.blockList) == 4  # Only 2 of the 20 blocks are valid

"Success?"

'Success?'

Now, we just need to connect our two functions `new_certificate(certificate)` and `new_block(block)` to the network in order to receive information from other nodes and ensure that everyone has the same blockchain.

***

<font color="7777aa">In your `BlockchainNode` class, rewrite the function `receive_object_from_node(obj, nodeIdentifier)` as in the example with `ExampleNode`: distribute the objects received from other nodes according to their type. Keep in mind that you will only receive objects of type `Certificate` and `Block`.</font>

***
***

To conclude these three tutorials, the following cell simulates 3 honest nodes synchronizing their blockchain. Our 4 favorite humans (Alice, Bob, Delphine, and Charlie the fraudulent) will add certificates to it. To diversify the identity of the forgers, the 3 nodes will stake an identical amount of tokens in the first block.

<font color="7777aa">The goal of this simulation is to validate the synchronization of the blockchain between the 3 nodes. To ensure that the 3 blockchains are identical, what simple test can you perform? Complete the function `are_nodes_synchronized(node1, node2, node3)` so that it returns `True` if the three nodes have exactly the same blockchain and `False` otherwise.</font>

In [14]:
from stake import StakingOperation

# Restart the network
Node.reset_network()

# Add 3 nodes

walletNode1 = defaultForgerWallet
node1 = BlockchainNode(walletNode1, proofOfStake)

walletNode2 = Wallet()
node2 = BlockchainNode(walletNode2, proofOfStake)

walletNode3 = Wallet()
node3 = BlockchainNode(walletNode3, proofOfStake)

nodes = [node1, node2, node3]

# 5 staking operations to start second block
# Each node will have 2 tokens staked after second block is forged

staking1 = StakingOperation(walletNode1.publicKey, 2)
walletNode1.sign(staking1)
node1.new_certificate(staking1)

staking2 = StakingOperation(walletNode2.publicKey, 1)
walletNode2.sign(staking2)
node2.new_certificate(staking2)

staking3 = StakingOperation(walletNode2.publicKey, 1)
walletNode2.sign(staking3)
node2.new_certificate(staking3)

staking4 = StakingOperation(walletNode3.publicKey, 1)
walletNode3.sign(staking4)
node3.new_certificate(staking4)

staking5 = StakingOperation(walletNode3.publicKey, 1)
walletNode3.sign(staking5)
node3.new_certificate(staking5)

# We will generate 100 certificates for our 4 humans
# Charlie's ones will all be invalid
# Every certificate is randomly given to one node

for i in range(20):
    selectedHuman = humans[i % len(humans)]
    if selectedHuman == Charlie:
        certificate = Certificate(Alice.publicKey)
    else:
        certificate = Certificate(selectedHuman.publicKey)
    selectedHuman.sign(certificate)
    nodes[int(random() * len(nodes))].new_certificate(certificate)
    
# Here, write a small test to verify that all blockchains (one for each node) are synchronized.
    
def are_nodes_synchronized(node1, node2, node3):
    # Rewrite the following function to test if all nodes are synchronized
    # ###########
    print(node1.blockchain.blockList)
    print(node2.blockchain.blockList)
    print(node3.blockchain.blockList)
    if node1.blockchain.get_latest_block().hash() == node2.blockchain.get_latest_block().hash() == node3.blockchain.get_latest_block().hash():
        
        return True
    else:
        return False
    # ###########
    
    
assert are_nodes_synchronized(node1, node2, node3)

"Assignment completed, good work!"

[<block.Block object at 0x00000237EA563260>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>]
[<block.Block object at 0x00000237EA563A10>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>]
[<block.Block object at 0x00000237EA563980>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>]


'Assignment completed, good work!'

### Bonus — Late to the Party

In our simulation, our three nodes are created simultaneously at the birth of the blockchain. Thus, when a block is forged, it is sent simultaneously to the three nodes and we never have indexing problems.

However, in reality, it doesn't work that smoothly. Some nodes may join the blockchain well after the first block has been forged. So, this new node, which only has the genesis block (index 0), will reject all the blocks relayed to it because all the other nodes are already on block index 10, or 100, etc.

To do things properly, the following process should be included in our `BlockchainNode` class:
* When receiving a block, if its index is higher than the index of our last block + 1, it may mean we are behind compared to all other nodes in the network. In this case, we set this block aside until we have the missing blocks.
* After setting a block aside, we send a request to everyone to receive the missing blocks by index.
* If we receive a request for missing blocks, we send back to the requester the blocks we have among those requested.
* Whenever we forge/receive a block, we check if one of the blocks set aside can be added following it. If so, we repeat this step.

<font color="77aa77">***Example:*** Nodes 1, 2, and 3 all have the same blockchain composed of 10 blocks. A new node joins the network: node 4. It only has the genesis block, index 0. Node 2 forges block index 11: it then transmits it to 1, 3, and 4. Nodes 1 and 3 accept it, while 4 cannot accept it because it is waiting for block index 1. Realizing its delay, it broadcasts a request to all nodes asking for blocks 1 to 10, and it sets aside block 11. Nodes 1, 2, and 3 hear its request and send blocks 1 to 10. Node 4 receives them, integrates them into its blockchain, and can thus add block 11 to them.</font>

To achieve this result, you will necessarily need to send and receive an additional type of object via your portal: the block request. To keep your implementation as simple as possible, make requests for one block at a time: you can thus reduce the request to a single integer representing the index of the block to be requested. Thus, enhance your `receive_object_from_node` function to take into account integers.

I recommend creating a new class to manage the set-aside blocks. However, don't worry about sending too many requests and overloading the network with information, as long as your code works, I will be happy.

**You will probably need to restart the entire notebook once to make the following cell work.**

In [15]:
# Let's add a 4th node to the network

walletNode4 = Wallet()
node4 = BlockchainNode(walletNode4, proofOfStake)

# Then let's redistribute 10 certificates to trigger a block forge

for i in range(10):
    selectedHuman = humans[i % len(humans)]
    if selectedHuman == Charlie:
        certificate = Certificate(Alice.publicKey)
    else:
        certificate = Certificate(selectedHuman.publicKey)
    selectedHuman.sign(certificate)
    nodes[int(random() * len(nodes))].new_certificate(certificate)
    
# We should have 4th node synchronized with the others
    
assert are_nodes_synchronized(node1, node2, node4)

"Congratz for finishing this difficult bonus!"

[<block.Block object at 0x00000237EA563260>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>, <block.Block object at 0x00000237EA58C9E0>, <block.Block object at 0x00000237EA42A7B0>]
[<block.Block object at 0x00000237EA563A10>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>, <block.Block object at 0x00000237EA58C9E0>, <block.Block object at 0x00000237EA42A7B0>]
[<block.Block object at 0x00000237EA45CB60>, <block.Block object at 0x00000237EA58CB30>, <block.Block object at 0x00000237EA705AF0>, <block.Block object at 0x00000237EA58C9E0>, <block.Block object at 0x00000237EA42A7B0>]


'Congratz for finishing this difficult bonus!'